In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import datetime
import geocoder
import os
from apiclient import discovery
from httplib2 import Http
import oauth2client
import io
from googleapiclient.http import MediaIoBaseDownload
from oauth2client import file, client, tools
%matplotlib inline
ox.config(log_console=True, use_cache=True)

Setting up Google Drive API [Tutorial](https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c)

In [2]:
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

In [3]:
cities = ['Batroun', 'Cap-Haitien', 'Lima', 'Montevideo', 'Province of Palawan', 'Sfax', 'Sousse', 'Vinh Long']

In [4]:
def list_ids(response):
    d={}
    page_token =None
    
    while True:
        for file in response.get('files', []):
            d[file.get('name')] = file.get('id')
            print ('Found file: %s (%s)' % (file.get('name'), file.get('id')))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    
    return d

In [6]:
def get_files_from_drive():
    d1={}
    ##
    # To access data from 'shared with me' folder, replace q with 
    #"sharedWithMe and name= = '{city}' and mimeType = 'application/vnd.google-apps.folder'".format(city=i)
    ##
    for i in cities:
        response = DRIVE.files().list(q="name = '{city}' and mimeType = 'application/vnd.google-apps.folder'".format(city=i),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()

        di = list_ids(response)    
        print('di', di ,'\n')

        response_ = DRIVE.files().list(q="'{}' in parents".format(di['{}'.format(i)]),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        d_ = list_ids(response_)
        print('d_', d_ ,'\n')
        for j in d_.keys():
            file_id = d_[j]
            request = DRIVE.files().get_media(fileId=file_id)
            # replace the filename and extension in the first field below
            try:
                os.mkdir('drive_data/{}'.format(i))
            except FileExistsError:
                pass
            except FileNotFoundError:
                os.mkdir('drive_data')

            s = j.split('.')
            try:
                ext=s[1]
            except IndexError:
                ext=None

            if ext is None and '_g' not in j:
                try:
                    os.mkdir('drive_data/{city}/{base}'.format(city=i, base=j))
                except FileExistsError:
                    pass
            else:
                fh = io.FileIO('drive_data/{city}/{file}'.format(city=i, file=j), mode='w')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                try:
                    status, done = downloader.next_chunk()
                    print("Download %d%%." % int(status.progress() * 100))
                except:
                    response_1 = DRIVE.files().list(q="'{}' in parents".format(d_[j]),
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
                    d1 = list_ids(response_1)
                    print('d1', d1, '\n')

                    for k in d1.keys():
                        file_id = d1[k]
                        request = DRIVE.files().get_media(fileId=file_id)
                        # replace the filename and extension in the first field below
                        try:
                            os.mkdir('drive_data/{city}/{base}'.format(city=i, base=j))
                        except FileExistsError:
                            pass
                        fh = io.FileIO('drive_data/{city}/{base}/{child}.{ext}'.format(city=i, base=j, child=k[:-4], ext=k[-3:]), mode='w')
                        downloader = MediaIoBaseDownload(fh, request)
                        done = False
                        while done is False:
                            status, done = downloader.next_chunk()
                            print("Download %d%%." % int(status.progress() * 100))
